In [1]:
# Imports
import numpy as np
import optuna
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from hftbacktest import Recorder
from hftbacktest.stats import LinearAssetRecord

from src.strategies import glft

# Data Input paths
preprocessed_data_path = "../data/daily_processed"
daily_eod_snapshots = "../data/snapshots" # EOD = End Of Day 

C:\Users\codri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Contract-specific constants (change to match the instrument)
TICK_SIZE = 0.05          # min price increment
LOT_SIZE  = 10            # contracts per lot
NOTIONAL  = 1.0           # linear P&L multiplier (1 for USD-quoted futures)

# Taken from https://support.deribit.com/hc/en-us/articles/25944746248989-Fees
MAKER_FEE = -0.0001   
TAKER_FEE =  0.0005    

In [3]:
from src.strategies.glft import gridtrading_glft_mm
day_start = 2
day_end = 22


def objective(trial):
    data = []
    latencies = []
    for i in range(day_start, day_end):
        day = str(i) if i > 9 else "0" + str(i)
        day_file = f"../data/daily_processed/deribit_eth_perp_2025-01-{day}.npz"
        day_latency = f"../data/latencies/latency_2025-01-{day}_latency.npz"
        data.append(day_file)
        # latencies.append(day_latency)
    day_start_str = str(day_start - 1) if day_start > 10 else "0" + str(day_start - 1)
    eod = np.load(f"../data/snapshots/deribit_eth_perp_2025-01-{day_start_str}_eod.npz")['data']

    gamma = trial.suggest_float("gamma", 0.01, 0.1)
    delta = trial.suggest_float("delta", 0.5, 2)
    adj1 = trial.suggest_float("adj1", 0.01, 2)
    adj2 = trial.suggest_float("adj2", 0.01, 1)
    max_position = trial.suggest_int("max_position", 1, 100)
    asset = (
    BacktestAsset()
        .data(data)
        .initial_snapshot(eod)
        .linear_asset(1.0)
        # .intp_order_latency(latencies, True)
        .constant_latency(0, 0) # Constant latency model (nanoseconds) values inspired from https://roq-trading.com/docs/blogs/2023-01-12/deribit/
        .risk_adverse_queue_model()
        # .power_prob_queue_model(2.0)
        .no_partial_fill_exchange()
        .trading_value_fee_model(MAKER_FEE, TAKER_FEE)
        .tick_size(0.01) # Tick size of this asset: minimum price increasement
        .lot_size(1) # Lot size of this asset: minimum trading unit
        # .roi_lb(0.0) # Sets the lower bound price for the range of interest in the market depth.
        # .roi_ub(3000.0) # Sets the upper bound price for the range of interest in the market depth.
        .last_trades_capacity(10000)
    )
    
    hbt = HashMapMarketDepthBacktest([asset])
  
    n_trading_days = day_end - day_start
    recorder = Recorder(1, n_trading_days* 1_000_000)
    
    gridtrading_glft_mm(hbt, recorder.recorder, n_trading_days, gamma, delta, adj1, adj2, max_position)
    hbt.close()

    stats = LinearAssetRecord(recorder.get(0)).stats()
    return stats.summary(pretty=True)['Return'][0]

In [4]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=42) # ensures reproducibility
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2025-05-17 00:55:37,441] A new study created in memory with name: no-name-989e5358-1042-4f85-a661-e4347e1cd2cb
[I 2025-05-17 01:02:23,711] Trial 0 finished with value: 753.6101489999893 and parameters: {'gamma': 0.04370861069626263, 'delta': 1.9260714596148742, 'adj1': 1.4666679442046962, 'adj2': 0.6026718993550663, 'max_position': 16}. Best is trial 0 with value: 753.6101489999893.
[I 2025-05-17 01:13:31,281] Trial 1 finished with value: 1191.8996200000006 and parameters: {'gamma': 0.02403950683025824, 'delta': 0.5871254182522991, 'adj1': 1.733690530092121, 'adj2': 0.6051038616257767, 'max_position': 71}. Best is trial 1 with value: 1191.8996200000006.
[I 2025-05-17 01:22:33,681] Trial 2 finished with value: 93.97072199998274 and parameters: {'gamma': 0.011852604486622221, 'delta': 1.9548647782429915, 'adj1': 1.6665608551928393, 'adj2': 0.2202157195714934, 'max_position': 19}. Best is trial 1 with value: 1191.8996200000006.
[I 2025-05-17 01:34:08,368] Trial 3 finished with value: 1

In [5]:
study.best_params

{'gamma': 0.012605087973827138,
 'delta': 0.584825015105819,
 'adj1': 1.2602649003885378,
 'adj2': 0.3718907708217256,
 'max_position': 42}

In [6]:
MAKER_FEE = 0.0000
sampler = TPESampler(seed=42) # ensures reproducibility
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2025-05-17 08:18:57,678] A new study created in memory with name: no-name-bbd34e0d-f9e3-4d51-b449-15b1aa648383
[I 2025-05-17 08:23:07,942] Trial 0 finished with value: 232.95500000000357 and parameters: {'gamma': 0.04370861069626263, 'delta': 1.9260714596148742, 'adj1': 1.4666679442046962, 'adj2': 0.6026718993550663, 'max_position': 16}. Best is trial 0 with value: 232.95500000000357.
[I 2025-05-17 08:27:15,929] Trial 1 finished with value: -1506.740000000018 and parameters: {'gamma': 0.02403950683025824, 'delta': 0.5871254182522991, 'adj1': 1.733690530092121, 'adj2': 0.6051038616257767, 'max_position': 71}. Best is trial 0 with value: 232.95500000000357.
[I 2025-05-17 08:31:25,775] Trial 2 finished with value: -1546.0500000000347 and parameters: {'gamma': 0.011852604486622221, 'delta': 1.9548647782429915, 'adj1': 1.6665608551928393, 'adj2': 0.2202157195714934, 'max_position': 19}. Best is trial 0 with value: 232.95500000000357.
[I 2025-05-17 08:35:33,947] Trial 3 finished with valu

In [7]:
study.best_params

{'gamma': 0.04370861069626263,
 'delta': 1.9260714596148742,
 'adj1': 1.4666679442046962,
 'adj2': 0.6026718993550663,
 'max_position': 16}